# Optimizing a production system

This tutorial will guide you through the optimization functionalities of `prodsys` to optimize the configuration of a production system. With the `prodsys.optimization` package, we can utilize meta-heuristics and mathematical optimization for this task. All algorithms can be conviently used with the `prodsys.models` API. 

For this example, we will use a production system which we will load from a json-file, which can be found in the examples folder of [prodsys' github page](https://github.com/sdm4fzi/prodsys). Download it and store it in the same folder as this notebook. Load the configuration and run a simulation with the following commands:

Let's start at first by loading our production system:

In [1]:
import prodsys

production_system = prodsys.adapters.JsonProductionSystemAdapter()	
production_system.read_data('example_configuration.json')

runner = prodsys.runner.Runner(adapter=production_system)
runner.initialize_simulation()
runner.run(20000)
runner.print_results()

100%|██████████| 20000/20000 [00:06<00:00, 2919.88it/s]



------------- Throughput -------------

              Output  Throughput
Product_type                    
Product_1        158    1.040803
Product_2        113    0.744371
Product_3        154    1.014453
------------- WIP -------------

Product_type
Product_1    18.744155
Product_2    18.725161
Product_3    17.993283
Total        53.351869
Name: WIP, dtype: float64

------------- Throughput time -------------

Product_type
Product_1    568.297738
Product_2    722.492383
Product_3    567.056246
Name: Throughput_time, dtype: float64

------------- Resource states -------------

                    time_increment  resource_time  percentage
Resource Time_type                                           
R1       PR            6487.179816   19993.737408   32.446059
         SB            9402.143058   19993.737408   47.025440
         ST            2907.339730   19993.737408   14.541252
         UD            1197.074804   19993.737408    5.987249
R2       PR            9517.090415   19993.

As already concluded in the seccond tutorial, this production system configuration is not capable of satisfying the arrival processes of the products and, thus, having very high WIP and throughput time values. In order to satify the product needs of our customers, we want to find a configuration with the `prodsys.optimization` package. However, for starting optimization, we also need to provide an optimization scenario, that models constraints, options, information and the objectives. Let's start by creating the constraints of the scenario with the `prodsys.models` API:

In [2]:
from prodsys.models import scenario_data

constraints = scenario_data.ScenarioConstrainsData(
    max_reconfiguration_cost=100000,
    max_num_machines=8,
    max_num_processes_per_machine=2,
    max_num_transport_resources=2
)

As you can see, the constraints consist of the maximum cost for reconfigruation and the maximumm number of machines, processes per machine and transport resources. Next, we define the options of our scenario for optimization:

In [3]:
positions = [[x*4, y*4] for x in range(4) for y in range(10)]
options = scenario_data.ScenarioOptionsData(
    transformations=[scenario_data.ReconfigurationEnum.PRODUCTION_CAPACITY],
    machine_controllers=["FIFO", "SPT"],
    transport_controllers=["FIFO", "SPT_transport"],
    routing_heuristics=["shortest_queue"],
    positions=positions
)

We specify in the scenario options the transformations that can be performed by the optmizer, which control policies and routing heuristics are available and what kind of positions are available to place resources. By choosing the transformation `PRODUCTION_CAPACITY`, the optimizer can add, remove or move production resources from the system or processes from single production resources.

At last, we need to specify our info for optimization:

In [4]:
info = scenario_data.ScenarioInfoData(
    machine_cost=40000,
    transport_resource_cost=20000,
    process_module_cost=4000,
    time_range=24*60
)

The scenario info contains information about the cost for machines, transport resources and process modules. Additionally, we specify a time range. This value is the time used for evalutation of created configurations during simulation. Since many evaluations are performed during optimization, this parameter can significantly influence the optimmization Time. For now, we specified it to one day. Lastly we can define the objectives used for optimization:

In [5]:
from prodsys.models.performance_indicators import KPIEnum
objectives = [scenario_data.Objective(
    name=KPIEnum.OUTPUT
)]

TODO

In [6]:
scenario = scenario_data.ScenarioData(
    constraints=constraints,
    options=options,
    info=info,
    objectives=objectives
)
production_system.scenario_data = scenario

TODO mkae API for optimization new to use these objects

In [7]:
from prodsys.optimization import evolutionary_algorithm

hyper_parameters = evolutionary_algorithm.EvolutionaryAlgorithmHyperparameters(
    seed=0,
    number_of_generations=10,
    population_size=10,
    mutation_rate=0.2,
    crossover_rate=0.1,
    number_of_processes=4
)
evolutionary_algorithm.optimize_configuration(
    production_system,
)